In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

In [2]:
in_data = pd.read_csv('./Dataset/BP_Dataset.csv')
in_data.head()

,Age(year),Gender,Height(cm),Weight(kg),BMI(kg/m^2),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),BP,cholesterol,gluc,smoke,alco,active,Target
0,50,2,168,62.0,21.967120,110,80,90.000000,1,1,0,0,1,Normal
1,55,1,156,85.0,34.927679,140,90,106.666667,3,1,0,0,1,Stage2
2,51,1,165,64.0,23.507805,130,70,90.000000,3,1,0,0,0,Stage1
3,48,2,169,82.0,28.710479,150,100,116.666667,1,1,0,0,1,Stage1
4,47,1,156,56.0,23.011177,100,60,73.333333,1,1,0,0,0,Normal


In [3]:
in_data = shuffle(in_data)
X = in_data.drop(['Target'], axis = 1)
X = np.array(X)
Y = np.array(in_data['Target'])
X[:10], Y[:10]

(array([[ 61.        ,   2.        , 159.        ,  66.        ,
          26.10656224, 120.        ,  80.        ,  93.3333332 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           1.        ],
        [ 60.        ,   2.        , 161.        ,  74.        ,
          28.54828132, 110.        ,  80.        ,  89.9999999 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           1.        ],
        [ 45.        ,   1.        , 160.        ,  86.        ,
          33.59375   , 130.        ,  90.        , 103.3333332 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           1.        ],
        [ 63.        ,   1.        , 156.        ,  76.        ,
          31.22945431, 150.        , 100.        , 116.6666665 ,
           3.        ,   1.        ,   1.        ,   0.        ,
           0.        ],
        [ 58.        ,   1.        , 168.        ,  61.        ,
          21.61281179, 120.        ,  80.        ,  93.3333

In [4]:
l_encode = LabelEncoder()
l_encode.fit(Y)
Y = l_encode.transform(Y)
Y = to_categorical(Y)
Y

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [5]:
train_x, test_x, train_y, test_y = train_test_split(X,Y, test_size = 0.3, random_state = 0)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((39200, 13), (39200, 4), (16800, 13), (16800, 4))

In [6]:
in_dim = len(in_data.columns)-1

model = Sequential()
model.add(Dense(8, input_dim = in_dim, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_x, train_y, epochs = 15, batch_size = 5)
scores = model.evaluate(test_x, test_y)

for i, m in enumerate(model.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

Epoch 1/15
7840/7840 [==============================] - 13s 2ms/step - loss: 0.6932 - accuracy: 0.7121
Epoch 2/15
7840/7840 [==============================] - 11s 1ms/step - loss: 0.1690 - accuracy: 0.9436
Epoch 3/15
7840/7840 [==============================] - 11s 1ms/step - loss: 0.0760 - accuracy: 0.9775
Epoch 4/15
7840/7840 [==============================] - 12s 2ms/step - loss: 0.0602 - accuracy: 0.9827
Epoch 5/15
7840/7840 [==============================] - 16s 2ms/step - loss: 0.0594 - accuracy: 0.9836
Epoch 6/15
7840/7840 [==============================] - 32s 4ms/step - loss: 0.0516 - accuracy: 0.9849
Epoch 7/15
7840/7840 [==============================] - 28s 4ms/step - loss: 0.0514 - accuracy: 0.9854
Epoch 8/15
7840/7840 [==============================] - 21s 3ms/step - loss: 0.0468 - accuracy: 0.9866
Epoch 9/15
7840/7840 [==============================] - 13s 2ms/step - loss: 0.0442 - accuracy: 0.9872
Epoch 10/15
7840/7840 [==============================] - 11s 1ms/step - l

In [7]:
test_size = 10
pred_probs = model.predict(test_x[:test_size])
pred = np.argmax(pred_probs, axis=1)
pred_ = l_encode.inverse_transform(pred)

true_y = l_encode.inverse_transform(np.argmax(test_y[:test_size], axis=1))

for i, j in zip(pred_, true_y):
    print("Predicted: {}, True: {}".format(i, j))


1/1 [==============================] - 0s 183ms/step
Predicted: Stage2, True: Stage2
Predicted: Normal, True: Normal
Predicted: Elevated, True: Elevated
Predicted: Normal, True: Normal
Predicted: Normal, True: Normal
Predicted: Elevated, True: Elevated
Predicted: Elevated, True: Elevated
Predicted: Stage1, True: Stage1
Predicted: Elevated, True: Elevated
Predicted: Elevated, True: Elevated


In [8]:
# Save the trained model weights to a file
model.save('new_model.h5')

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
import joblib

# Save the LabelEncoder's classes_ attribute to a file
joblib.dump(l_encode.classes_, 'label_encoder_classes.pkl')



['label_encoder_classes.pkl']